### ***Document Understanding & Interpretation w/ `zyx`***

In [1]:
! pip install --upgrade --quiet zyx

In [1]:
import zyx

The `zyx.read()` function is capable of reading most document types, URLs to documents, as well as lists of URL's and paths.

In [2]:
# Lets try reading a paper on Arxiv
document = zyx.read("https://arxiv.org/pdf/2407.21787")

The read function returns a `Document` object by default. A `Document` object contains two main fields: `content` and `metadata`. Lets try inspecting these two fields now.

In [3]:
print("Metadata: ", document.metadata)
print("Content: ", document.content[:200]) # We'll print the first 200 characters.

Metadata: 
{'file_name': '2407.21787', 'file_type': 'application/pdf', 'file_size': 955583}

Content:  Large Language Monkeys: Scaling Inference Compute
with Repeated Sampling
Bradley Brown∗†‡, Jordan Juravsky∗†, Ryan Ehrlich∗†, Ronald Clark‡, Quoc V. Le§,
Christopher R´ e†, and Azalia Mirhoseini†§
†De

In [4]:
# Alternatively, the `zyx.read()` function can also return just a string, but for this example we will be specifically
# using the `Document` object.

# document = zyx.read("https://arxiv.org/pdf/2407.21787", output = str)

One of the neat things about the `Document` object is that it is directly queryable with LLMs to achieve `single-document QA`.

In [5]:
# This supports all arguments of `zyx.completion()`, so it can be used with all LiteLLM compatible models.
response = document.query(
    "In 2-3 sentences, summarize the key findings of this paper.",
    model = "gpt-4o-mini"
)

In [6]:
# The response is in the standard OpenAI `ChatCompletion` or LiteLLM `ModelResponse` format, lets print out just the message content.
print(response.choices[0].message.content)

The paper demonstrates that scaling inference compute through repeated sampling significantly improves the coverage
of language models across various tasks, with increases in problem-solving success rates directly correlating to 
the number of samples generated. By amplifying weaker models with repeated sampling, the authors found that models 
like DeepSeek-Coder can outperform stronger models like GPT-4 despite lower individual capabilities, making this 
approach more cost-effective.

We can expand further with our `Document` now, by adding it to a collection. `zyx` contains three built-in collections: `Sql` or an SQLModel BM25 based document store, `VectorStore` a Qdrant based vector store, and `Rag` a collection that uses a combination of both

In [7]:
# Lets try using the `VectorStore` collection now.
collection = zyx.VectorStore(
    collection_name = "document-understanding",
    location = ":memory:"
)

2024-09-26 12:39:49.369 | INFO     | zyx.lib.data.vector_store:_create_collection:93 - Collection 'document-understanding' does not exist. Creating it now.
2024-09-26 12:39:49.370 | INFO     | zyx.lib.data.vector_store:_create_collection:102 - Collection 'document-understanding' created successfully.


In [8]:
# .add() also supports adding strings of text
# collection.add("This is a test document")

collection.add(document)

2024-09-26 12:39:54.779 | INFO     | zyx.lib.data.vector_store:add:181 - Successfully added 14 points to the collection.


> Although using document stores is the standard for document understanding with LLMs when working with multiple documents. LLM's now have a high enough context window that they can often perform single-document QA without the need for a document store. Hence, using `zyx.read()`, & `document.query()` are often enough for simple use cases.

Lets now query our collection.

In [9]:
results = collection.completion(
    "What did the author do in their research?"
)

print(results.choices[0].message.content)

2024-09-26 12:39:54.791 | INFO     | zyx.lib.data.vector_store:completion:303 - Initial messages: What did the author do in their research?


In their research, the authors explored the concept of scaling inference compute in large language models (LLMs) by
utilizing repeated sampling to improve performance on various tasks. They conducted experiments to analyze how 
increasing the number of generated samples affected the coverage (the fraction of problems solved by any attempt) 
and precision (the ability to identify correct solutions from generated samples).

Key findings from their research include:

1. Significant improvement in coverage was observed by increasing the number of samples. For example, the coverage 
for certain tasks increased dramatically with more attempts, showing log-linear scalability.
  
2. In domains with automatic verification tools, such as coding and formal proofs, this repeated sampling approach 
translated directly into improved performance metrics. For instance, they were able to achieve a higher 
problem-solving rate compared to existing state-of-the-art models.

3. The research highlighted the potential for cost-effective solutions using less powerful models amplified by 
repeated sampling. They found that using multiple samples from a less expensive model could outperform a single 
attempt from a more advanced model.

4. The authors also identified challenges in precision when selecting correct solutions from a large set of 
generated samples, noting that common methods like majority voting often plateaued in effectiveness beyond a 
certain number of samples.

Overall, the research emphasizes the benefits of repeated sampling as a strategy for enhancing the capabilities of 
LLMs in various reasoning tasks and suggests future research directions to improve sample verification methods.